In [1]:
# Instalación de Librerias

!pip install transformers
!pip install gradio
!pip install Pillow
!pip install accelerate -U
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
# Importamos Paquetes y Configuracion de Modelos

import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import pipeline
import gradio as gr
from PIL import Image


# Modelos:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
traductor = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [3]:
# Función `generar_descripcion`

def generar_descripcion(imagen):
    """
    Genera una descripción en español para una imagen dada.
    """
    # Paso 1: Procesar la imagen y generar la descripción en inglés
    # 'processor' prepara la imagen para el modelo (la convierte a un formato que entiende).
    inputs = processor(imagen, return_tensors="pt")

    # 'model.generate' es el corazón del captioning.
    # Le pasamos los inputs procesados y le pedimos que genere hasta 50 palabras.
    out = model.generate(**inputs, max_length=50)

    # 'processor.decode' convierte los números que generó el modelo en texto legible.
    # 'skip_special_tokens=True' saca tokens que no son parte de la descripción.
    descripcion_en = processor.decode(out[0], skip_special_tokens=True)

    # Paso 2: Traducir la descripción al español
    # Usamos nuestro "traductor" para convertir la descripción de inglés a español.
    # [0]['translation_text'] es para extraer el texto de la traducción.
    descripcion_es = traductor(descripcion_en)[0]['translation_text']

    # Paso 3: Devolver los resultados
    # La función devuelve un diccionario con las descripciones en ambos idiomas.
    return {
        "Descripción (English)": descripcion_en,
        "Descripción (Español)": descripcion_es
    }

In [4]:
# Interfaz con Gradio

# Para que nuestro sistema sea fácil de usar para cualquiera,
# vamos a crear una interfaz web simple con Gradio.
# Esto nos permite subir una imagen y ver los resultados en el navegador.

# Mirá cómo armamos la interfaz:
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🖼️ Generador Automático de Descripciones de Imágenes
    ¡Subí una imagen y vas a ver cómo el sistema le genera una descripción automáticamente en español!
    """)

    with gr.Row():
        imagen_input = gr.Image(type="pil", label="Subí tu imagen acá")

    with gr.Row():
        boton_analizar = gr.Button("Generar Descripción")

    with gr.Row():
        salida = gr.JSON(label="Resultados")

    boton_analizar.click(
        fn=generar_descripcion,
        inputs=imagen_input,
        outputs=salida
    )

    gr.Markdown("""
    ### 📝 Instrucciones para Usar la Interfaz:
    1.  **Subí una imagen:** Usá el campo de arriba que dice "Subí tu imagen acá". Podés arrastrar y soltar una foto, o **clickear** para seleccionarla desde tu computadora.
    2.  **Generar la descripción:** Una vez que la imagen está cargada, **hacé clic** en el botón "Generar Descripción".
    3.  **Observá los resultados:** Vas a ver un cuadro con formato JSON que te va a mostrar la descripción en inglés y la traducción al español.

    """)


demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://22fb0248650b7e8847.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
